In [15]:
import sys
import os
import h5py
from collections import Counter
from progressbar import *
import re
import numpy as np
import h5py
from scipy import signal
import matplotlib
from repeating_classifier_training_utils import *
from functools import reduce
from tqdm import tqdm

# Add the path to the parent directory to augment search for module
par_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

if par_dir not in sys.path:
    sys.path.append(par_dir)
%load_ext autoreload
%matplotlib inline
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
veto_path = '/fast_scratch/WatChMaL/data/IWCDmPMT_4pm_full_tank_ODveto.h5'
odv_file = h5py.File(veto_path,'r')
odv_info = {}
for key in odv_file.keys():
    odv_info[key] = np.array(odv_file[key])

In [17]:
veto_dic = {}
for i, veto in tqdm(enumerate(odv_info['veto'])):
    veto_dic[odv_info['root_files'][i], odv_info['event_ids'][i]] = veto

9000000it [00:14, 619640.77it/s]


In [18]:
event_data_file = h5py.File('/fast_scratch/WatChMaL/data/IWCDmPMT_4pi_full_tank_pointnet.h5','r')

dset_root_files = event_data_file['root_files']
dset_event_ids = event_data_file['event_ids']

In [5]:
idx_file = np.load('/fast_scratch/WatChMaL/data/IWCD_fulltank_300_pe_idxs.npz', allow_pickle=True)
test_idxs = idx_file['test_idxs']
train_idxs = idx_file['train_idxs']
val_idxs = idx_file['val_idxs']

In [21]:
test_veto = np.array([veto_dic[(dset_root_files[idx], dset_event_ids[idx])] for idx in tqdm(test_idxs)])

100%|██████████| 3351020/3351020 [20:23<00:00, 2739.64it/s]


In [22]:
train_veto = np.array([veto_dic[(dset_root_files[idx], dset_event_ids[idx])] for idx in tqdm(train_idxs)])

100%|██████████| 4188774/4188774 [25:00<00:00, 2792.05it/s]


In [23]:
val_veto = np.array([veto_dic[(dset_root_files[idx], dset_event_ids[idx])] for idx in tqdm(val_idxs)])

100%|██████████| 837754/837754 [05:07<00:00, 2721.21it/s]


In [28]:
assert val_veto.shape[0] == val_idxs.shape[0]
assert train_veto.shape[0] == train_idxs.shape[0]
assert test_veto.shape[0] == test_idxs.shape[0]

In [29]:
post_veto_train_idxs=train_idxs[np.where(train_veto==0)[0]]
post_veto_test_idxs=test_idxs[np.where(test_veto==0)[0]]
post_veto_val_idxs=val_idxs[np.where(val_veto==0)[0]]

In [30]:
np.savez('/fast_scratch/WatChMaL/data/IWCD_fulltank_300_post_veto_pe_idxs.npz', train_idxs=post_veto_train_idxs, test_idxs=post_veto_test_idxs, val_idxs=post_veto_val_idxs)

In [31]:
event_data_file.close()